# Stream-Graph Tutorial

We will first load a very common dataset known as [CollegeMsg](https://snap.stanford.edu/data/CollegeMsg.html)

This dataset is comprised of private messages interactions commited on an online social network at the University of California, Irvine. Users could search the network for others and then initiate conversation based on profile information. An edge $(u, v, t)$ means that user $u$ sent a private message to user $v$ at time $t$.

In [1]:
import pandas as pd
df = pd.read_csv('collegeMsg.txt', sep=' ', names=['u', 'v', 'ts'])
df.head()

,u,v,ts
0,1,2,1082040961
1,3,4,1082155839
2,5,2,1082414391
3,6,7,1082439619
4,8,7,1082439756


Which consists of 59835 interactions

In [2]:
df.shape[0]

59835

## Stream-Graph Library

What the above dataset expresses is what in our library's terminology is called the **TemporalLinkSet**.  
Here we will initiliaze a temporal-link-set of instantaneous links, in discrete-time.  
We don't know if our dataset contains duplicates so we want to signify that to our class-constructor, while we don't want to apply a further sorting to our values.

In [3]:
from stream_graph import ITemporalLinkSetDF
ls = ITemporalLinkSetDF(df, no_duplicates=False, discrete=True, sort_by=None)

Our first operation we can have, is calculating the number of nodes that appear inside our temporal linkset.

In [4]:
ls.nodeset.size

1899

Where not all their interactions are distinct.

In [5]:
ls.size

59798

### Measures
So in order to see which users **send** most-messages, we can jus calculate and sort the out-degree.

In [6]:
DF = pd.DataFrame(list(ls.degree_of(direction='out')), columns=['user', 'out-degree'])
DF = DF.set_index('user').sort_values(by=['out-degree'], ascending=False)
DF.head(5)

,out-degree
user,
9,1091
323,1011
12,993
103,739
105,685


And let's do the same for those users who **receive** the most-messages, we can just calculate and sort the in-degree.

In [7]:
DF = pd.DataFrame(list(ls.degree_of(direction='in')), columns=['user', 'in-degree']).set_index('user')
DF.sort_values(by=['in-degree'], ascending=False).head(5)

,in-degree
user,
1624,558
323,534
32,500
103,440
372,427


In order to see which communication between two users lasted the most, what you need is known as the duration of the link.  
So let's calculate the durations.

In [8]:
data = [(a, b, d) for ((a, b), d) in ls.duration_of(direction='both')]
DF = pd.DataFrame(data, columns=['user-a', 'user-b', 'duration']).set_index(['user-a', 'user-b'])

And plot the *max-5*.

In [9]:
DF.sort_values(by=['duration'], ascending=False).head(5)

,,duration
user-a,user-b,
1168,1624,184
398,1624,166
12,1312,164
454,1043,154
105,1624,141


## Measures of centrality
### Closeness

A simple measure of centrality is known as closeness.
In the standard graph, closeness measures the amount that a node is 'closer' to all the others in the graph.
A definition of closeness inside graph theory of a node $u$ is:
$$C(u)=\sum_{v \neq u} \frac{1}{d(u, v)}$$
where, $d(u, v)$ is the shortest-path between $u$ and $v$.

Stream-Graphs, are graphs that have a temporal direction and as so the closeness of a node $u$, **at time** $t$ is defined as:
$$C_{t}(u)=\sum_{v \neq u} \frac{1}{d_{t}(u, v)}$$
where, $d_{t}(u, v)$ is the shortest-path between $u$ and $v$, at time $t$.

So let's the calculate timestamp where the closeness of each node becomes maximum (without taking direction into account).  
Execution time in this case, is a little bit higher.

In [10]:
DF = pd.DataFrame([(u, mxc, ts) for u, (mxc, ts) in ls.closeness(t='max', direction='both')], columns=['user', 'max-closeness', 'ts']).set_index('user')

and print the top 20 most-significant values:

In [11]:
DF.sort_values(by=['max-closeness', 'ts'], ascending=False).head(20)

,max-closeness,ts
user,,
1713,38.111374,1089632769
172,38.111279,1089632769
652,38.111261,1089632769
176,38.111259,1089632769
242,38.111254,1089632769
1190,38.111254,1089632769
199,38.111254,1089632769
26,38.111253,1089632769
252,38.111253,1089632769


Now let's plot the closeness-profile of user 1713.

In [12]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [13]:
t, value = zip(*((t, v) for t, v in ls.closeness(u=1713, direction='both') if v>.0001))

In [14]:
from bokeh.plotting import figure, show

p = figure()
p.scatter(t, value, line_color=None)

show(p)

And maybe the closeness at that point in time.

In [15]:
DF = pd.DataFrame([(u, round(cl, 2)) for u, cl in ls.closeness(t=1089632769, direction='both') if round(cl, 2) != .0], columns=['user', 'closeness'])

In [16]:
def to_set(x):
    return set(x)

DF.groupby('closeness')['user'].agg({'size': len, 'group': to_set}).sort_values(by=['closeness', 'size'], ascending=False)

/home/mouchette/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


,size,group
closeness,,
38.11,14,"{288, 640, 3, 1190, 199, 172, 652, 176, 1713, ..."
20.44,18,"{32, 1185, 1154, 68, 72, 840, 1662, 302, 846, ..."
20.11,1,{2}
9.78,19,"{1281, 645, 782, 408, 800, 1440, 1315, 810, 69..."
9.44,3,"{249, 1237, 569}"
1.61,19,"{1415, 394, 794, 545, 1189, 681, 687, 1335, 82..."
0.28,5,"{1412, 1317, 933, 42, 173}"


## Ego-Betweenness
Another measure of centrality is known as ego-betweennesss. Initially the ego-betweeness of a node $e$ is defined as:
$$ C(e)=\sum_{i, j \in \mathcal{N}_{e} \times \mathcal{N}_{e}} \frac{g_{i j}(e)}{g_{i j}} $$
where $g_{i j}$ stands for the number of shortest paths between $i$ and $j$, and $g_{i j}(e)$ for those that pass through $e$.  


Now concerning **temporal** graphs the ego-betweeness of a node $e$ at time $t$ is defined as:
$$C(e, \tau)=\sum_{i, j \in \mathcal{N}_{e} \times \mathcal{N}_{e}} \frac{p_{i j}(e, \tau)}{p_{i j}(\tau)}$$
where $p_{ij}(\tau)$ is the number of most recent paths of length at most 2 from $i$ to $j$ at time $\tau$ and $p_{ij}(e, \tau)$ is the number of these paths that go through $e$.

So let's find out how much is the betweenes-centrality of all nodes at this obscure timestamp 1089632769.  
Execution time in this case, will be much higher.
#### Note, that in order to calculate the centrality at a certain time, our algorithm has it that this should be first calculated for all timestamps

In [ ]:
DF = pd.DataFrame([(u, bt) for u, bt in ls.ego_betweeness(t=1089632769, direction='both')], columns=['user', 'betweeness'])

In [ ]:
DF.groupby('betweeness')['user'].agg({'size': len, 'group': to_set}).sort_values(by=['betweeness', 'size'], ascending=False)

## Cliques
A clique in a classic graph is defined as a set of nodes, where there exists a link between all pairs of nodes (subgraphs of density 1).

![image info](./graph-clique.png)

In a link-stream of continuous duration, a clique is defined as subset of the timeset and a subset of the nodes, where links exist between everybody.

![image info](./link-stream-clique.png)

### Delta-Cliques
Next we want to calculate what is called "Delta-Cliques".

The logic behind this is the following. We add $\pm \frac{\delta}{2}$ duration to each interaction and we calculate under this setting what are the maximum cliques, that is the maximum time where all nodes in a subset of all the users are interacting with each other. This can signify a group communication under a certain period.

In [ ]:
# Delta cliques inside a twenty minute period
mxc = ls.get_maximal_cliques(direction='both', delta=1200)

And now let's sort and print the ten biggest delta-cliques.

In [ ]:
# Efficient: whenever an actor exists it connects with all the other actors existing
sorted_cliques = sorted(list(mxc), key=lambda x:(len(x[0]), x[1][1] - x[1][0]), reverse=True)
sorted_cliques[:10]